In [1]:
import pandas as pd
import numpy as np

## Metadata

In [143]:
PATH_TRAIN = "source/evo_train.csv"
PATH_CATS = "source/categories.csv"

In [144]:
target_label = u'GROUP_ID'
pred_labels = u'NAME'

## Preparing data ##

In [145]:
dtrain = pd.read_csv(PATH_TRAIN)
cats = pd.read_csv(PATH_CATS) 

In [5]:
target = dtrain[target_label]
preds = dtrain[pred_labels]

### Partition

In [6]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    preds, target, 
    test_size=0.33, 
    random_state=42)

### Functions

In [7]:
def check_accuracy(clf, x, y, x_test, y_test):
    clf.fit(x, y)
    t = clf.predict(x_test)
    from sklearn.metrics import accuracy_score
    print "accuracy_score: %f" % accuracy_score(y_test, t)
    
# def check_accuracy(clf, X_train, y_train, X_test, y_test):
#     clf.fit(x, y)
#     t = clf.predict(x_test)
#     from sklearn.metrics import accuracy_score
#     print "accuracy_score: %f" % accuracy_score(y_test, t)

## Model

### Baseline ###

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
baseline_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB())])
check_accuracy(baseline_clf, X_train, y_train, X_test, y_test)

accuracy_score: 0.812249


### Best model

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
clf_v4 = Pipeline([('vect', CountVectorizer()),
                   ('tfidf', TfidfTransformer(use_idf=False)),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))])
check_accuracy(clf_v4, X_train, y_train, X_test, y_test)

accuracy_score: 0.892210


## Best features

In [53]:
# SelectKBest

from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline


clf_v5 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer(use_idf=False)),
                   ('ch2', SelectKBest(chi2, k=14000)),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))
                  ])

# t = clf_v5.fit_transform(X_train, y_train)
check_accuracy(clf_v5, X_train, y_train, X_test, y_test)
print cross_val_score(clf_v5, X_train, y_train, scoring='accuracy', cv=4, n_jobs=-1)

accuracy_score: 0.897418
[ 0.89463008  0.89284188  0.90088742  0.89845923]


## BEST MODEL

In [97]:
# SelectFromModel

from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score


clf_v5 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer(use_idf=False)),
                   ('select', SelectFromModel(LogisticRegression(C=0.01, penalty="l2", dual=False), threshold="0.01*mean")),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-4, 
                                         n_iter=5, 
                                         random_state=42))
                  ])

# t = clf_v5.fit_transform(X_train, y_train)
check_accuracy(clf_v5, X_train, y_train, X_test, y_test)
print cross_val_score(clf_v5, X_train, y_train, scoring='accuracy', cv=4, n_jobs=-1)

accuracy_score: 0.929912
[ 0.92003843  0.91858974  0.92494387  0.92403167]


In [175]:
# SelectFromModel

from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score


clf_v5 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer(use_idf=False)),
                   ('select', SelectFromModel(LogisticRegression(C=0.01, penalty="l2", dual=False), 
                                              threshold="0.01*mean")),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-4, 
                                         n_iter=50, 
                                         random_state=42))
                  ])

# t = clf_v5.fit_transform(X_train, y_train)
check_accuracy(clf_v5, X_train, y_train, X_test, y_test)
print cross_val_score(clf_v5, X_train, y_train, scoring='accuracy', cv=4, n_jobs=-1)

accuracy_score: 0.930889
[ 0.92078574  0.92008547  0.92729605  0.92563664]


In [176]:
# SelectFromModel

from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score


clf_v5 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer(use_idf=False)),
                   ('select', SelectFromModel(LogisticRegression(C=0.01, penalty="l2", dual=False), 
                                              threshold="mean")),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-4, 
                                         n_iter=50, 
                                         random_state=42))
                  ])

# t = clf_v5.fit_transform(X_train, y_train)
check_accuracy(clf_v5, X_train, y_train, X_test, y_test)
print cross_val_score(clf_v5, X_train, y_train, scoring='accuracy', cv=4, n_jobs=-1)

accuracy_score: 0.909244
[ 0.89601794  0.89487179  0.89971132  0.90370212]


In [51]:
%%time
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

counter = CountVectorizer()
t = counter.fit_transform(X_train)

lr = LogisticRegression(C=10000, penalty="l2", dual=False)
selector = SelectFromModel(lr, threshold="mean")
selector.fit(t, y_train)

X_new = selector.transform(t)
print t.shape, X_new.shape

clf_v5 = Pipeline([('vect', counter),
                   ('select', selector),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))
                  ])

check_accuracy(clf_v5, X_train, y_train, X_test, y_test)
print cross_val_score(clf_v5, X_train, y_train, scoring='accuracy', cv=4, n_jobs=-1)

(37426, 37474) (37426, 12152)
accuracy_score: 0.904958
[ 0.89815309  0.89797009  0.90452261  0.90177616]
Wall time: 3min 53s


In [174]:
%%time
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

counter = CountVectorizer()
t = counter.fit_transform(X_train)

lr = LogisticRegression(C=10000, penalty="l2", dual=False)
selector = SelectFromModel(lr, threshold="mean")
selector.fit(t, y_train)

X_new = selector.transform(t)
print t.shape, X_new.shape

clf_v5 = Pipeline([('vect', counter),
                   ('select', selector),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))
                  ])

# check_accuracy(clf_v5, X_train, y_train, X_test, y_test)
from sklearn import metrics
clf_v5.fit(X_train, y_train)
print(metrics.classification_report(y_test, clf_v5.predict(X_test)))

             precision    recall  f1-score   support

          6       0.73      0.94      0.82      1955
          7       0.92      0.96      0.94      2031
          9       0.97      0.87      0.92       198
         12       0.89      0.71      0.79       185
         14       0.89      0.89      0.89      2002
         15       0.94      0.91      0.92        66
         16       0.94      0.74      0.83       203
         17       0.94      0.91      0.92       215
         18       0.95      0.72      0.82       205
         19       0.91      0.96      0.94       192
         20       0.93      0.76      0.84       217
         21       0.93      0.71      0.81       196
         22       0.90      0.71      0.79       197
         24       0.87      0.91      0.89      1929
         25       0.96      0.99      0.97       985
         26       0.90      0.87      0.89      1004
         27       0.90      0.48      0.63       221
         28       0.92      0.77      0.83   

In [57]:
counter = CountVectorizer()
t = counter.fit_transform(X_train)

lr = LogisticRegression(C=10000, penalty="l2", dual=False)
selector = SelectFromModel(lr, threshold="mean")
selector.fit(t, y_train)

X_new = selector.transform(t)
print t.shape, X_new.shape

(37426, 37474) (37426, 12152)


In [87]:
clf = SGDClassifier(loss='hinge', penalty='l2',
                    alpha=1e-3, n_iter=5,
                    random_state=42)
clf.fit(X_new, y_train)
print cross_val_score(clf, X_new, y_train, scoring='accuracy', cv=4, n_jobs=-1)

[ 0.90381125  0.90202991  0.90773014  0.90530708]


In [96]:
clf = SGDClassifier(loss='hinge', penalty='l2',
                    alpha=1e-4, n_iter=5,
                    random_state=42)
clf.fit(X_new, y_train)
print cross_val_score(clf, X_new, y_train, scoring='accuracy', cv=4, n_jobs=-1)

[ 0.92089249  0.92094017  0.92473003  0.92306869]


In [81]:
print len(clf.classes_)
print clf.
print clf.coef_.shape

29
None
(29L, 12152L)


In [64]:
print sum(selector.get_support())
pd.Series(counter.vocabulary_)[selector.get_support()].to_csv('selected_log_reg.csv', sep='\t', encoding="UTF-8")


12152


In [117]:
t = pd.DataFrame(clf.coef_.transpose())
s = pd.DataFrame(pd.Series(counter.vocabulary_)[selector.get_support()].reset_index())
s.columns = ['ngram', 'index']

In [124]:
ngrams = pd.concat([s,t], axis=1)

In [129]:
ngrams.columns = ['ngram', 'index'] + [str(x) for x in clf.classes_]

In [135]:
ngrams.to_csv('ngram_coeffs.csv', encoding='UTF-8')

In [163]:
clf.classes_

array([ 6,  7,  9, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27,
       28, 29, 30, 32, 34, 35, 36, 37, 38, 52, 59, 60], dtype=int64)

In [171]:
group_id = 35
colname = str(group_id)
print cats[cats.GROUP_ID==group_id].NAME.iloc[0]
ngrams.sort_values(by=[colname], ascending=False)[['ngram',colname]]

Пиво, слабоалкогольные напитки


,ngram,35
8638,пиво,2.179356
3696,балтика,2.019891
8637,пивной,2.019891
9821,светлое,2.019891
5478,жигулевское,1.966736
4161,бутылка,1.913581
10730,темное,1.860426
6764,кулер,1.860426
7960,нефильтрованное,1.807271
10016,сидр,1.754116


In [49]:
# RFE

from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.feature_selection import RFE
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.pipeline import Pipeline


clf_v5 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer(use_idf=False)),
                   ('select', RFE(LogisticRegression(C=0.01, penalty="l2", dual=False), 
                                  n_features_to_select=10000,
                                  step=1000)),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))
                  ])

# t = clf_v5.fit_transform(X_train, y_train)
check_accuracy(clf_v5, X_train, y_train, X_test, y_test)
print cross_val_score(clf_v5, X_train, y_train, scoring='accuracy', cv=4, n_jobs=-1)

accuracy_score: 0.903928


KeyboardInterrupt: 

In [31]:
%%time
counter = CountVectorizer()

t, test, ty, testy = X_train, X_test, y_train, y_test
# train_test_split(
#     X_train, y_train, 
#     test_size=0.33, 
#     random_state=17)

counter.fit(t)

t = counter.transform(t)
test = counter.transform(test)

nrow = t.shape[0]
ncol = t.shape[1]
print nrow, ncol

import random
from deap import creator, base, tools, algorithms

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

# toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("attr_bool", lambda: np.random.random_sample() < 0.95)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=ncol)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

clf = SGDClassifier(loss='hinge', 
                    penalty='l2',
                    alpha=1e-3, 
                    n_iter=5,
                    random_state=42)

def get_accuracy(clf, x, y, x_test, y_test):
    clf.fit(x, y)
    t = clf.predict(x_test)
    from sklearn.metrics import accuracy_score
    res = accuracy_score(y_test, t)
    print "accuracy_score: %f; shape %d" % (res, x.shape[1])
    return res        

def evalAccuracy(individual):
    indices = np.array(individual).astype(bool)
    return get_accuracy(clf, t[:,indices], ty, test[:,indices], testy),

def mutate(individual):
    size = int(sum(individual) * 0.01)
    indices = np.random.randint(ncol, size=size)
    for i in indices:
        individual[i] = 0
    return individual,

def mate(ind1, ind2):
    return (creator.Individual(np.array(ind1).astype(bool) & np.array(ind2).astype(bool)),
           creator.Individual(np.array(ind1).astype(bool)))

toolbox.register("evaluate", evalAccuracy)
toolbox.register("mate", mate)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=300)

NGEN=40
try:
    for gen in range(NGEN):
        offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
        fits = toolbox.map(toolbox.evaluate, offspring)
        for fit, ind in zip(fits, offspring):
            ind.fitness.values = fit
        population = toolbox.select(offspring, k=len(population))
finally:
    top10 = tools.selBest(population, k=10)
    print top10[0]

37426 37474
accuracy_score: 0.895953; shape 35670
accuracy_score: 0.895682; shape 35652
accuracy_score: 0.897364; shape 35563
accuracy_score: 0.901866; shape 35645
accuracy_score: 0.888901; shape 35690
accuracy_score: 0.900673; shape 35579
accuracy_score: 0.893783; shape 35562
accuracy_score: 0.899588; shape 35629
accuracy_score: 0.900564; shape 35264
accuracy_score: 0.895465; shape 35271
accuracy_score: 0.899208; shape 35303
accuracy_score: 0.901432; shape 35651
accuracy_score: 0.898828; shape 35616
accuracy_score: 0.892427; shape 35626
accuracy_score: 0.902083; shape 35616
accuracy_score: 0.900727; shape 35585
accuracy_score: 0.896279; shape 33906
accuracy_score: 0.901432; shape 35617
accuracy_score: 0.890745; shape 33494
accuracy_score: 0.897201; shape 35572
accuracy_score: 0.896658; shape 35599
accuracy_score: 0.897364; shape 35305
accuracy_score: 0.899045; shape 35595
accuracy_score: 0.897743; shape 35539
accuracy_score: 0.899859; shape 35247
accuracy_score: 0.901703; shape 35654


KeyboardInterrupt: 

In [27]:
rfe.get_support()

array([ True,  True, False, ...,  True, False, False], dtype=bool)

In [21]:
t.shape[1]

(37426, 37474)

### GA

In [12]:
import random
from deap import creator, base, tools, algorithms

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalOneMax(individual):
    return sum(individual),

toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=300)

NGEN=40
for gen in range(NGEN):
    offspring = algorithms.varAnd(population, toolbox, cxpb=0.5, mutpb=0.1)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
    population = toolbox.select(offspring, k=len(population))
top10 = tools.selBest(population, k=10)

In [54]:
counter = CountVectorizer()
t = counter.fit_transform(X_train)
k=0.01 * t.shape[1]
k=100
chi = SelectKBest(chi2, k=k)
t =chi.fit_transform(t, y_train)
print t.shape
print t.shape[1]

(37426, 100)
100


In [79]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel

counter = CountVectorizer()
t = counter.fit_transform(X_train)

lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(t, y_train)
model = SelectFromModel(lsvc)
X_new = model.fit_transform(t)
X_new.shape

ValueError: bad input shape ()

In [76]:
pd.Series(counter.vocabulary_)[model.get_support()]

0mg             380
10              396
100г            453
100мл           461
150мл          1391
1кг            1851
1л             1857
20             1896
30             2799
30ml           2876
30мл           2888
37             3241
3mg            3333
3мг            3368
40             3389
5л             4345
75             4874
adalya         5835
blue           6549
compact        7181
eleaf          7821
fanatik        8049
giay           8480
gsh            8641
iphone         9062
mg            10092
red           11324
silver        11862
smok          11945
strobbs       12204
              ...  
тени          33511
термос        33560
тетрадь       33612
товар         33733
томат         33768
тон           33785
трости        34076
тумба         34149
турка         34166
туф           34182
туфли         34185
тушь          34196
унитаза       34475
услуги        34588
филе          34932
фонарь        35142
хлеб          35553
цифра         35946
чай           35997


In [55]:
print len(chi.get_support())
print len(counter.vocabulary_)

37474
37474


In [56]:
pd.Series(counter.vocabulary_)[chi.get_support()]

00               0
5л            4345
6s            4707
bb            6345
bg            6440
fanatik       8049
ip            9050
iphone        9062
vandoren     12741
алк          13706
альта        13794
анальная     13892
белое        14893
ботинки      15442
брелок       15508
вешалка      16298
винный       16413
вино         16414
водка        16645
волос        16752
вох          16841
гавриш       17081
гер          17271
горшок       17678
детские      18412
для          18653
жен          19297
жидкость     19368
заказ        19547
защитное     19724
             ...  
саксофона    30835
сапоги       30983
сверло       31100
светлое      31117
свободная    31204
свободной    31205
свободные    31206
семена       31363
серьги       31504
сигареты     31565
собак        32113
сопрано      32266
столовое     32780
сувенир      32942
сумка        32978
таб          33236
табл         33243
тенора       33519
тетрадь      33612
трости       34076
тумба        34149
туфли       

## Batch

In [10]:
from sklearn.utils import resample

n_samples = 10000

Bpreds, Btarget = resample(preds, target, 
                           replace=False,
                           n_samples=n_samples,
                           random_state=17)

In [11]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    Bpreds, Btarget, 
    test_size=0.33, 
    random_state=42)

### Baseline ###

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
baseline_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', MultinomialNB())])
check_accuracy(baseline_clf, X_train, y_train, X_test, y_test)

accuracy_score: 0.721515


### Best model

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
clf_v4 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer(use_idf=False)),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))])
check_accuracy(clf_v4, X_train, y_train, X_test, y_test)

accuracy_score: 0.853636


In [14]:
import re
def replace_digits(line):
    pat = ur'[\d]{2,1000}([\W][\d]+)*'
    repl = " "
    return re.sub(pat, repl, line)

In [15]:
repl = [" ", " %DIGIT% "]
max_df = [0.5, None]
use_idf = [True, False]

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
clf_v4 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer()),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))])
check_accuracy(clf_v4, X_train.apply(lambda x: replace_digits(x)), y_train, X_test.apply(lambda x: replace_digits(x)), y_test)

accuracy_score: 0.857879


In [18]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
def normal_form(word):
    return morph.parse(word)[0].normal_form

def normal_form_line(line):    
    return u' '.join([normal_form(unicode(x, 'utf-8')) for x in line.split()])

ImportError: No module named pymorphy2

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
clf_v4 = Pipeline([('vect', CountVectorizer()),
#                    ('tfidf', TfidfTransformer()),
                   ('clf', SGDClassifier(loss='hinge', 
                                         penalty='l2',
                                         alpha=1e-3, 
                                         n_iter=5, 
                                         random_state=42))])
check_accuracy(clf_v4, X_train.apply(lambda x: replace_digits(x)).apply(lambda x: normal_form_line(x)), y_train, X_test.apply(lambda x: replace_digits(x)).apply(lambda x: normal_form_line(x)), y_test)

NameError: global name 'normal_form_line' is not defined

In [ ]:
vect =  CountVectorizer()
t = vect.fit_transform(X_train.apply(lambda x: replace_digits(x)))
vocab = pd.Series(vect.vocabulary_)

In [ ]:
vocab.count()

In [ ]:
pd.DataFrame(t.todense())

In [ ]:
vocab.to_csv("vocab_batch.csv", encoding="UTF-8", sep='\t')